In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from utils.utils_io import webp_loader, pfm_loader, plot_and_save_figure, plot_figure
from utils.misc import Rescale, ToTensor, PadSampleToBatch

In [ ]:
from model.model_sceneflow import StereoNet
# from model.model_kitti_2015 import StereoNet
# from model.model_kitti_2012 import StereoNet

In [ ]:
path_to_left_rgb_image_file = \
r'fill_in'
path_to_right_rgb_image_file = \
r'fill_in'
path_to_left_disp_file = \
r'fill_in'


In [ ]:
# Load in the image pair as numpy uint8 arrays
sample = {'left': webp_loader(path_to_left_rgb_image_file),
          'right': webp_loader(path_to_right_rgb_image_file)
          }

# sample = {'left': utils.image_loader(path_to_left_rgb_image_file),
#           'right': utils.image_loader(path_to_right_rgb_image_file)
#           }


In [ ]:
# disp_gt = utils.pfm_loader(path_to_left_disp_file)
disp_gt, scale = pfm_loader(path_to_left_disp_file)
print(type(disp_gt), disp_gt.max(), disp_gt.min())

In [ ]:
# plt.imshow(disp_gt[0])
plt.imshow(disp_gt, cmap='plasma')

In [ ]:
# Transform the single image pair into a torch.Tensor then into a
# batch of shape [batch, channel, height, width]
transformers = [
    ToTensor(), 
    PadSampleToBatch(),
    Rescale()
    ]
    
for transformer in transformers:
    sample = transformer(sample)


In [ ]:
print(type(sample))
print(sample.keys())

print(type(sample['left']))
print(sample['left'].shape)
print(sample['left'].max())
print(sample['left'].min())

In [ ]:
plt.imshow(sample['left'].squeeze().moveaxis(0, 2))

In [ ]:
base_model = StereoNet.load_from_checkpoint(
    r'fill_in'
    )

In [ ]:
model = StereoNet.load_from_checkpoint(

    # BEST SCENE FLOW
    # r'fill_in'

    # BEST KITTI 2015
    # r'fill_in'

    # BEST KITTI 2012
    r'fill_in'

    )

In [ ]:
# Set the model to eval and run the forward method without tracking gradients
model.eval()
with torch.no_grad():
    batched_prediction = model(sample)

In [ ]:
base_model.eval()
with torch.no_grad():
    base_prediction = base_model(sample)

In [ ]:
# Remove the batch diemnsion and switch back to channels last notation
single_prediction = batched_prediction[0].numpy()  # [batch, ...] -> [...]
single_prediction = np.moveaxis(single_prediction, 0, 2)  # [channel, height, width] -> [height, width, channel]

single_base_prediction = base_prediction[0].numpy()  # [batch, ...] -> [...]
single_base_prediction = np.moveaxis(single_base_prediction, 0, 2)  # [channel, height, width] -> [height, width, channel]

In [ ]:
print(type(base_prediction))
print(base_prediction.shape)
print(base_prediction.max())
print(base_prediction.min())

print()

print(type(batched_prediction))
print(batched_prediction.shape)
print(batched_prediction.max())
print(batched_prediction.min())


In [ ]:
print(type(single_base_prediction))
print(single_base_prediction.shape)
print(single_base_prediction.max())
print(single_base_prediction.min())

print()

print(type(single_prediction))
print(single_prediction.shape)
print(single_prediction.max())
print(single_prediction.min())

print()


print(type(disp_gt))
print(disp_gt.shape)
print(disp_gt.max())
print(disp_gt.min())

In [ ]:
plt.imshow(single_base_prediction, cmap='plasma')

In [ ]:
plt.imshow(single_prediction, cmap='plasma')

In [ ]:
print(type(sample['left']))
print(type(sample['right']))
print(type(disp_gt), disp_gt.shape, disp_gt.max(), disp_gt.min())
print(type(single_prediction), single_prediction.shape, single_prediction.max(), single_prediction.min())

single_prediction = single_prediction.squeeze()

single_prediction = np.where(disp_gt > 0, single_prediction, 0)

figure = plot_and_save_figure(sample['left'], sample['right'], disp_gt, single_prediction)

In [ ]:
print((disp_gt - single_prediction).max())

In [ ]:
plt.imshow(np.abs(disp_gt - single_prediction), cmap='plasma')

In [ ]:
print(type(sample['left']))
print(type(sample['right']))
print(type(disp_gt), disp_gt.max(), disp_gt.min())
print(type(single_base_prediction), single_base_prediction.max(), single_base_prediction.min())

# figure = plot_figure(sample['left'], sample['right'], disp_gt[0], single_base_prediction)
figure = plot_figure(sample['left'], sample['right'], disp_gt, single_base_prediction)